# Harmonic Balance using exponential expansions

In [ ]:
using LinearAlgebra
using BoundaryValueDiffEq
using SparseArrays
using CairoMakie
using ModelingToolkit
using MethodOfLines
using DifferentialEquations
using NonlinearSolve
using DomainSets
using FFTW
using GLMakie

Define the variable parameters for the wave equation

In [ ]:
gamma = 0.0;
omega = 40.0;
gamma3 = 1.0;
g0 = 9.80665; # m / s^2
height = 5; # m

Define the constants specific to the discretizations

In [ ]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
N = 1000;
H = 2; # number of harmonics
order = 2;
stepx = (xright-xleft)/N;

## **Try out different methods to do the Symbolic computations on higher harmonics**

Using Python's famous SymPy

In [ ]:
import SymPy as sp

@time begin
    x = sp.symbols("x")
    f = sp.cos(5x)^4
    
    fs = sp.sympy.fourier_series(f, (x, -sp.PI, sp.PI))
end
println(fs.truncate(n=10))

Using Julia's ApproxFun.jl

In [ ]:
import ApproxFun as AF

@time begin
    # Define on CosSpace (cosine series only)
    f = AF.Fun(x -> cos(5x)^4, AF.CosSpace())
    
    coeffs = AF.coefficients(f)
end

println("Cosine Series:")
for (k, c) in enumerate(coeffs)
        if abs(c) > 1e-10
            println("$(c) * cos($((k-1))x)")
        end
end

## Use the SymPy method to start a potential Harmonic Balance

$$
u = \sum_{k=1}^H A_k \cos(k\omega t) + B_k \sin(k\omega t)
$$

In [ ]:
import ModelingToolkit as Model
import SymPy as sp
import Symbolics as Symb

In [ ]:
# Define symbolics
Model.@parameters x, t;
Model.@variables A(..), B(..);

u = A(x) * sin(omega*t) + B(x) * cos(omega*t)
Dx = Model.Differential(x)
Dt = Model.Differential(t)
y = Dt(Dt(u)) - g0*H*Dx(Dx(u)) + gamma*Dt(u) + gamma3*Dt(u)*Dt(u)*Dt(u)
y_exp = Symb.expand(Model.expand_derivatives(y))
println(y_exp)